In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.neighbors import NearestNeighbors

import warnings






In [6]:
def make_recommendation(Title, Rating, Genres):
    # Load the dataset
    movies_df = pd.read_csv('cleaned_movies_data.csv')
    
    # Define meta and feature columns
    meta_cols = ['Title', 'Rating', 'Genres']
    feature_cols = ['Release_Date', 'Rating', 'No_of_Persons_Voted', 'Duration', 'Genres']
    
    # Define preprocessing pipelines
    numeric_features = ['Rating', 'No_of_Persons_Voted']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    categorical_features = ['Genres']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )
    
    # Find the index of the movie based on the provided details
    try:
        movie_index = movies_df.loc[
            (movies_df.Title == Title) &
            (movies_df.Release_Date == Release_Date) &
            (movies_df.Rating == Rating) &
            (movies_df.Directed_by == Directed_by) &
            (movies_df.Genres == Genres)
        ].index[0]
    except IndexError:
        return f"No movie found with the provided details."

    # Prepare the features for modeling
    X = movies_df.loc[:, feature_cols]
    
    preprocessor.fit(X)
    X_preprocessed = preprocessor.transform(X)
    
    # Set k to a fixed number of neighbors (e.g., 5)
    k = 5
    model1 = NearestNeighbors(n_neighbors=k, metric='cosine')
    
    model1.fit(X_preprocessed)
    
    # Get features for the selected movie
    movie_features = X_preprocessed[movie_index].reshape(1, -1)
    
    distances, indices = model1.kneighbors(movie_features)
    
    # Get the recommended movies
    recommended_movies = movies_df.iloc[indices[0]]
    recommended_movies["distances"] = distances[0]
    
    # Sort by distance and return the recommendations
    recommended_movies = recommended_movies.sort_values(by="distances")
    
    return recommended_movies.to_dict(orient="records"), movies_df.loc[movie_index, meta_cols].to_dict()


In [9]:
Title = "The Man"
Rating = 9.2  # Ensure this matches the rating in your dataset
Genres = "History"  # Ensure this matches the genres in your dataset

response, metadata = make_recommendation(Title, Rating, Genres)

pd.DataFrame(response)

ValueError: too many values to unpack (expected 2)

In [ ]:
# Load the saved cosine similarity matrix
with open('cosine_similarity.pkl', 'rb') as f:
    loaded_cosine_sim = pickle.load(f)

# Load the saved movie data
with open('movie_data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)


In [ ]:
# Example movie title
recommended_movies = recommend_movies('Inception')
print(recommended_movies)


In [ ]:
def make_recommendation(Title, Release_Date, Rating, Directed_by, Genres):
    # Load the dataset
    movies_df = pd.read_csv('cleaned_movies_data.csv')
    
    meta_cols = ['Title', 'Directed_by']
    feature_cols = ['Release_Date', 'Rating', 'No_of_Persons_Voted', 'Duration', 'Genres']
    
    
    
    numeric_features = ['Rating', 'No_of_Persons_Voted']
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')), #?
        ('scaler', StandardScaler())])
    
    binary_features = []
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy = 'most_frequent', missing_values=pd.NA)),
        ('label', OrdinalEncoder())])
    
    categorical_features = ['Genres']
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy = 'most_frequent', missing_values=pd.NA)),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('binary', binary_transformer, binary_features),
            ('cat', categorical_transformer, categorical_features)])
    
    Title = df.loc[(df.Release_Date == Release_Date) & (df.Rating == Rating) & (df.Directed_By == Directed_By) & (df.Genres == Genres)] \
            .sort_values(by="Rating", ascending=False).movie_id.values[0]
    
    X = df.loc[:, feature_cols]
    
    preprocessor.fit(X)
    X_preprocessed = preprocessor.transform(X)
    
    k = Genres
    model1 = NearestNeighbors(n_neighbors=k, metric='cosine')
    
    model1.fit(X_preprocessed)
    
    movie_features = df.loc[df.movie_id == movie_id, feature_cols]
    movie_features_preprocessed = preprocessor.trasnform(movie_features)
    
    distances, indices = model1.kneighbors(movie_features_preprocessed)
    
    movies = df.iloc[indices[0]]
    movies["distances"] = distances[0]
    
    cols = movies.columns
    movies = movies.loc[:,cols]
    movies = movies.sort_values(by="distances")
    
    return movies.to_dict(orient="records")